In [31]:
import pennylane as qml
import numpy as np

# Define the quantum device 
device = qml.device("default.qubit", wires=1)

In [32]:
# Define quantum circuit for Alice's encoding
def alice_encode(bit, basis):
    @qml.qnode(device)
    def circuit():
        if bit == 1:
            qml.PauliX(wires=0)  # Apply X gate if the bit is 1
        if basis == 'S':
            qml.Hadamard(wires=0)  # Apply H gate for superposition
        return qml.state()  # Return the quantum state
    return circuit()

In [33]:
# Define quantum circuit for measurement (used by Eve and Bob)
def measure_qubit(basis):
    @qml.qnode(device)
    def circuit():
        if basis == 'S':
            qml.Hadamard(wires=0)  # Apply H gate for superposition measurement
        return qml.sample(wires=0)  # Measure the qubit with finite shots
    return circuit

In [34]:
# Alice's choices and bits
alice_bits = [0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
alice_choices = ['NS', 'NS', 'S', 'NS', 'S', 'NS', 'S', 'NS', 'NS', 'S']
encoded_qubits = []

In [35]:
# Alice encodes the qubits
for bit, basis in zip(alice_bits, alice_choices):
    encoded_qubits.append(alice_encode(bit, basis))

print("Alice created the key of 10 bits:", alice_bits)
print("Alice's superposition choices:", alice_choices)
print("Now, Alice sends her qubits to Bob >>>>>>>>>>>>>>>")

Alice created the key of 10 bits: [0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
Alice's superposition choices: ['NS', 'NS', 'S', 'NS', 'S', 'NS', 'S', 'NS', 'NS', 'S']
Now, Alice sends her qubits to Bob >>>>>>>>>>>>>>>


In [37]:
# Eve intercepts
print("***************** IF EVE INTERCEPTS! ******************")
eve_choices = ['S', 'NS', 'S', 'NS', 'NS', 'S', 'NS', 'S', 'S', 'NS']
eve_bits = []

# Eve measures and re-encodes the qubits
for state, basis in zip(encoded_qubits, eve_choices):
    @qml.qnode(device)
    def eve_measure_and_reencode():
        qml.QubitStateVector(state, wires=0)  # Initialize the state
        if basis == 'S':
            qml.Hadamard(wires=0)  # Apply H gate for superposition measurement
        return qml.expval(qml.PauliZ(0))  # Measure the expected value

    eve_measurement = eve_measure_and_reencode()
    eve_bit = 0 if eve_measurement > 0 else 1  # Interpret expval as a bit
    eve_bits.append(eve_bit)

    @qml.qnode(device)
    def reencode_qubit():
        if eve_bit == 1:
            qml.PauliX(wires=0)  # Apply X gate if the bit is 1
        if basis == 'S':
            qml.Hadamard(wires=0)  # Apply H gate for superposition
        return qml.state()

    # This encodes the qubit for Bob
    reencode_qubit()

print("Eve measured bits:", eve_bits)
print("Eve sends intercepted qubits to Bob >>>>>>>")

***************** IF EVE INTERCEPTS! ******************
Eve measured bits: [1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
Eve sends intercepted qubits to Bob >>>>>>>


In [40]:
# Define a device with analytic mode
device = qml.device("default.qubit", wires=1)  # Analytic mode is enabled by default

def measure_qubit(state, basis):
    @qml.qnode(device)
    def circuit():
        qml.QubitStateVector(state, wires=0)  # Initialize the state
        if basis == 'S':
            qml.Hadamard(wires=0)  # Apply H gate for superposition measurement
        # Use expected value to determine the bit
        return qml.expval(qml.PauliZ(wires=0))  # Measure along the Z-axis
    return circuit

# Bob's choices and measurement
bob_choices = ['S', 'NS', 'NS', 'NS', 'S', 'NS', 'NS', 'NS', 'S', 'S']
bob_bits = []

for state, basis in zip(encoded_qubits, bob_choices):
    # Measure the qubit and append the result
    expval = measure_qubit(state, basis)()
    bob_bit = 0 if expval > 0 else 1  # Convert expectation value to bit (0 for +1, 1 for -1)
    bob_bits.append(bob_bit)

print("Bob's superposition choices:", bob_choices)
print("Bob measured bits:", bob_bits)


Bob's superposition choices: ['S', 'NS', 'NS', 'NS', 'S', 'NS', 'NS', 'NS', 'S', 'S']
Bob measured bits: [1, 1, 1, 0, 0, 0, 1, 0, 1, 1]


In [41]:
# Key reconciliation
alice_key = []
bob_key = []

for a_choice, b_choice, a_bit, b_bit in zip(alice_choices, bob_choices, alice_bits, bob_bits):
    if a_choice == b_choice:
        alice_key.append(a_bit)
        bob_key.append(b_bit)

print("Alice's effective key:", alice_key)
print("Bob's effective key:", bob_key)
if alice_key == bob_key:
    print("SECURE! Keys match:", alice_key)
else:
    print("EVE DETECTED! Keys mismatch.")


Alice's effective key: [1, 0, 0, 0, 0, 1]
Bob's effective key: [1, 0, 0, 0, 0, 1]
SECURE! Keys match: [1, 0, 0, 0, 0, 1]


In [48]:
# WITHOUT EVE'S INTERCEPTION
print("\\nWITHOUT EVE'S INTERCEPTION\\n")
encoded_qubits = []
for bit, basis in zip(alice_bits, alice_choices):
    encoded_qubits.append(alice_encode(bit, basis))

bob_bits = []
for state, basis in zip(encoded_qubits, bob_choices):
    # Measure the qubit with the given state and basis
    bob_bit = int(measure_qubit(state, basis)())  # Pass both state and basis
    bob_bits.append(bob_bit)

\nWITHOUT EVE'S INTERCEPTION\n


In [49]:
alice_key = []
bob_key = []

for a_choice, b_choice, a_bit, b_bit in zip(alice_choices, bob_choices, alice_bits, bob_bits):
    if a_choice == b_choice:
        alice_key.append(a_bit)
        bob_key.append(b_bit)

print("Alice's effective key:", alice_key)
print("Bob's effective key:", bob_key)

if alice_key == bob_key:
    print("SECURE! Keys match:", alice_key)
else:
    print("EVE DETECTED! Keys mismatch.")

Alice's effective key: [1, 0, 0, 0, 0, 1]
Bob's effective key: [-1, 1, 0, 1, 1, 0]
EVE DETECTED! Keys mismatch.
